In [1]:
import numpy as np
import scipy.integrate as integrate
import scipy.special as special

import random
import os
import csv

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

import json

from matplotlib import pyplot as plt
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

$$
\sin(\pi x) e^{-\pi^2 t}
$$

In [7]:
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [11]:
# ============
# save and load: json
# ============

def save_json(save_path, data):
    assert save_path.split('.')[-1] == 'json'
    with open(save_path,'w+') as file:
        json.dump(data,file)

def load_json(file_path):
    assert file_path.split('.')[-1] == 'json'
    with open(file_path,'r') as file:
        data = json.load(file)
    return data

# ============
# save and load: csv
# ============

def save_csv(save_path, data):
    with open(save_path, "w+") as f:
        writer = csv.writer(f)
        writer.writerows(data)

def load_csv(file_path):
    string = []
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if i%2==0:
                string.append(row)
    ret = []
    for p in string:
        ret.append(np.array([float(i) for i in p]))
    return ret
    
# ==============
# save and load: normal
# ==============

def save_list(save_path, data):
    file = open(save_path, 'w+')
    for value in data:
        file.write(str(value)+" ")
    file.close()

# def load_list(file_path):

In [20]:
# =======
# OPTIONS
# =======

# ===========
# padding method
# ===========

def padding(starting_padding, original, end_padding):
    return np.hstack((starting_padding, original, end_padding)).tolist()

def zero_padding(original, num1, num2):
    starting_padding = [0 for i in range(num1)]
    end_padding = [0 for i in range(num2)]
    return padding(starting_padding, original, end_padding)

def border_padding(original, num1, num2):
    starting = original[0]
    ending = original[len(original)-1]
    starting_padding = [starting for i in range(num1)]
    end_padding = [ending for i in range(num2)]
    return padding(starting_padding, original, end_padding)

def recursive_padding(original, num1, num2):
    starting_padding = original[-num1:]
    end_padding = original[:num2]
    return padding(starting_padding, original, end_padding)

def random_padding(original, num1, num2):
    max_value = np.max(original)
    min_value = np.min(original)
    starting_padding = [random.randint(min_value, max_value) for i in range(num1)]
    end_padding = [random.randint(min_value, max_value) for i in range(num2)]
    return padding(starting_padding, original, end_padding)

In [17]:
# =======
# OPTIONS
# =======

# ==================
# analytical solution generator
# ==================

def gen_analytical(delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq):
    x = arange(xmin, xmax, delta_x)
    t = arange(tmin, tmax, delta_t)
    X,T = meshgrid(x, t) # grid of point
    solu = analytical_eq(X, T) # evaluation of the function on the grid
    return x, t, solu

def gen_analytical_averaged(delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq):
    x = arange(xmin, xmax, delta_x)
    t = arange(tmin, tmax, delta_t)
    X,T = meshgrid(x, t) # grid of point
    Z = analytical_eq(X, T) # evaluation of the function on the grid
    solu = []
    for zz in Z:
        solu_t = []
        for j in range(len(zz)-1):
            value = (1/2) * (zz[j] + zz[j+1])
            solu_t.append(value)
        solu.append(solu_t)
    return x, t, solu

def gen_analytical_cell_averaged(delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq):
    x = arange(xmin, xmax, delta_x)
    t = arange(tmin, tmax, delta_t)
    solu = []
    for ti in range(len(t)):
        solu_t = []
        for j in range(len(x)-1):
            value = integrate.quad(lambda x: analytical_eq(x, t[ti]), x[j], x[j+1])
            value = value[0] * (1/delta_x)
            solu_t.append(value)
        solu.append(solu_t)
    return x, t, solu

In [34]:
# =======
# OPTIONS
# =======

# ===========
# training set
# ===========

def get_trainingset_random(solu, num1, num2, padding, size):
    solu_padding = []
    pairs = []
    for item in solu:
        p = padding(item, num1, num2)
        solu_padding.append(p)
    for iteration in range(size):
        t_index = random.randint(0, len(solu_padding)-2)
        x_index = random.randint(0, len(p)-num1-num2-1)
        time = solu_padding[t_index]
        time_next = solu_padding[t_index+1]
        train = time[x_index: x_index+num1+num2+1]
        target = time_next[x_index+num1]
        pair = {'train': train, 'target': target}
        pairs.append(pair)
    return pairs
    
def get_trainingset_all(solu, num1, num2, padding):
    solu_cutted = solu[:-1]
    pairs = []
    for index, item in enumerate(solu_cutted):
        p = padding(item, num1, num2)
        p_next = padding(solu[index+1], num1, num2)
        for xi in range(len(p)-num1-num2-1):
            train = p[xi: xi+num1+num2+1]
            target = p_next[xi+num1]
            pair = {'train': train, 'target': target}
            pairs.append(pair)
    return pairs

# ===========
#  testing set
# ===========

def get_testingset_random(solu, num1, num2, padding, size):
    return get_trainingset_random(solu, num1, num2, padding, size)

def get_testingset_all(solu, num1, num2, padding):
    solu_cutted = solu[:-1]
    pairs = []
    for index, item in enumerate(solu_cutted):
        p = padding(item, num1, num2)
        p_next = padding(solu[index+1], num1, num2)
        pairs_t = []
        for xi in range(len(p)-num1-num2-1):
            train = p[xi:xi+num1+num2+1]
            target = p_next[xi+num1]
            pair = {'train': train, 'target': target}
            pairs_t.append(pair)
        pairs.append(pairs_t)
    return pairs

# x, t, solu = gen_analytical_cell_averaged(delta_x=1/20, delta_t=1/10, xmin=0, tmin=0, xmax=2, tmax=1, analytical_eq=heat_equ_analytical_solu)
# pairs = get_testingset_all(solu, 3, 3, recursive_padding)

In [39]:
# ==============
# FullconnectedResNet
# ==============

class FullconnectedResNet(nn.Module):
    def __init__(self, weight=1, i, o, layer_data, hasDropout, p, num):
        super(FullconnectedResNet, self).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module("linear_1", nn.Linear(i, layer_data[0]))
        if hasDropout:
            self.layers.add_module("dropout_1", nn.Dropout(p=p))
        self.layers.add_module("relu_1", nn.ReLU())
        for index in range(len(layer_data)-1):
            self.layers.add_module("linear_"+str(index+2), nn.Linear(layer_data[index], layer_data[index+1]))
            if hasDropout:
                self.layers.add_module("dropout_2", nn.Dropout(p=p))
            self.layers.add_module("relu_"+str(index+2), nn.ReLU())
        self.layers.add_module("linear_3"+str(len(layer_data)+1), nn.Linear(layer_data[len(layer_data)-1], o))
        if hasDropout:
            self.layers.add_module("dropout_3", nn.Dropout(p=p))
        self.layers.add_module("relu_"+str(len(layer_data)+1), nn.ReLU())
        self.num = num
        self.weight = weight
        
    def forward(self, x):
        output = self.layers(x)
        return output + self.weight * x[self.num]

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)
        
# ==========
# BidirectionRNN
# ==========

# class BidirectionRNN(nn.Module):

In [46]:
def printer_of_nn(train, target, loss, output):
    return 0

In [41]:
def json_config_generator(
    delta_x, delta_t, xmin, tmin, xmax, tmax, analytical_eq, gen_analytical_method, allTheTime
    num1, num2, padding, train_takeAll, eval_takeAll, size1, size2,
    layer_data, nn_model, optim, learning_rate, iteration, hasDropout, p, weight,
    loading, load_path,
    config_file):
    data_setting = {'delta_x': delta_x,'delta_t': delta_t,'xmin': xmin, 'tmin': tmin, 'xmax': xmax, 'tmax': tmax, 'PDE': str(analytical_eq), 'method': str(gen_analytical_method), 'allTheTime': allTheTime}
    pairs_setting = {'left': num1,'right': num2, 'dim': num1+num2+1, 'padding': str(padding), 'train_takeAll': train_takeAll, 'eval_takeAll': eval_takeAll, 'size_train': size1, 'size_eval': size2}
    nn_setting = {'model': str(nn_model), 'layer_data': layer_data, 'input': num1+num2+1, 'output': 1, 'hasDropout': hasDropout, 'dropout_rate': p, 'res_weight': weight}
    optim_setting = {'optimizer': str(optim), 'learning_rate': learning_rate}
    model_setting = {'nn_model': nn_setting, 'optim': optim_setting, 'iteration': iteration, 'is_loading': loading, 'loading_path': load_path}
    json_data = {
        'data_setting': data_setting,
        'pairs_setting': pairs_setting,
        'model_setting': model_setting
    }

In [47]:
def training_the_model(
    delta_x=1/20, delta_t=1/10, xmin=0, tmin=0, xmax=2, tmax=1, analytical_eq=heat_equ_analytical_solu, gen_analytical_method=gen_analytical_cell_averaged, allTheTime=False,
    num1=3, num2=3, padding=recursive_padding, train_takeAll=True, eval_takeAll=True, size1=0, size2=0,
    layer_data=[6, 6], nn_model=FullconnectedResNet, optim=optim.Adam, learning_rate=0.001, iteration=100, hasDropout=False, p=0.2, weight=1,
    loading=False, load_path="",
    folder="", config_file="", actaul_solu_file="",
    doEval=True, printTraining=False, printEval=False):
    # ===========
    # Prepare the data
    # ===========
    x, t, solu = gen_analytical_method(delta_x=delta_x, delta_t=delta_t, xmin=xmin, tmin=tmin, xmax=xmax, tmax=tmax, analytical_eq=analytical_eq)
    actual_solu = solu
    if not allTheTime:
        solu = solu[:2]
    # ===========
    # Prepare the pairs
    # ===========
    if train_takeAll:
        pairs = get_trainingset_all(solu, num1, num2, padding)
    else:
        pairs = get_trainingset_random(solu, num1, num2, padding, size)
    # =================
    # Set up model & optimizer
    # =================
    model = nn_model(weight=weight, i=num1+num2+1, o=1, layer_data=layer_data, hasDropout=hasDropout, p=p, num=num1)
    optimizer = optim(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    # ===========
    # Train the model
    # ===========
    model.train()
    list_of_losses_t = []
    list_of_outputs = []
    counter = 0
    for itera in range(iteration):
        for pair in pairs:
            train = torch.FloatTensor(pair["train"])
            target = torch.FloatTensor([pair["target"]])
            model.zero_grad()
            optimizer.zero_grad()
            output = model(train)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            list_of_losses_t.append(loss.item())
            list_of_output.append(output)
            if printTraining:
                printer_of_nn(train, target, loss, output)
            counter += 1
    # ===========
    # Prepare the pairs
    # ===========
    if eval_takeAll:
        eval_pairs = get_testingset_all(solu, num1, num2, padding)
    else:
        eval_pairs = get_testingset_random(solu, num1, num2, padding, size)
    # =============
    # Evaluate the model
    # =============
    model.eval()
    list_of_losses_e = []
    prediction = []
    

In [ ]:
json_file="config.json", solu_file="solu.csv", input_file="inputs.txt", loss_file="train_losses", outout_file="outputs.txt", model_file = "model")